In [106]:
import pandas as pd
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

pd_train = pd.read_json('https://raw.githubusercontent.com/mdemaster/w207_Final_Project/master/train.json', orient='columns')
pd_test = pd.read_json('https://raw.githubusercontent.com/mdemaster/w207_Final_Project/master/test.json', orient='columns')

np_test = np.array(test)
np_train = np.array(train)



In [105]:


X = np_train[:,6]
Y = np_train[:,22]
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print 'data shape: ', X.shape
print 'label shape:', Y.shape

l=len(X)
train_data, train_labels = X[:l/2], Y[:l/2]
dev_data, dev_labels = X[l/2:(3*l)/4], Y[l/2:(3*l)/4]
test_data, test_labels = X[(3*l)/4:], Y[(3*l)/4:]




data shape:  (4040,)
label shape: (4040,)


In [123]:
#Run initial vectorizer and fit_transform on train_data and find vocab size from shape attribute.
vect=CountVectorizer()
data=vect.fit_transform(train_data).toarray()
devdata=vect.transform(dev_data).toarray()


#Use np.where to binarize train and dev set where values above and below 0.5.
b=train_labels
trainlabels=np.where(b==True, 1, 0)

bl=dev_labels
devlabels=np.where(bl==True, 1, 0)

print 'Baseline Scores...'

#Run MultinomialNB Classifier
mnb_clf = Pipeline([('vect', CountVectorizer()), ('mnclf',MultinomialNB(alpha=0.01))])
mnb_clf = mnb_clf.fit(train_data, trainlabels)
pred = mnb_clf.predict(dev_data)
score1=metrics.accuracy_score(devlabels,pred)
print 'Naive Bayes Score:',score1

#Run Logistic Regression classifier
log_clf = Pipeline([('vect', CountVectorizer()),('lgclf', LogisticRegression(C=0.5))])
log_clf = log_clf.fit(train_data, trainlabels) 
pred = log_clf.predict(dev_data)        
score2= metrics.accuracy_score(devlabels,pred)
print 'Logistic Regression Score:',score2

Baseline Scores:
Naive Bayes Score: 0.683168316832
Logistic Regression Score: 0.69702970297
